In [13]:
import numpy as np
import circe as ci
import scanpy as sc
import scipy as sp
import warnings
warnings.filterwarnings('ignore')

In [14]:
regions = ['AMY','HIP','PFC']
celltypes = ['OPC-Oligo', 'Immune','Astro-Epen','Vascular']


In [15]:
region = 'AMY'
celltype = 'Vascular'
base_name = f"{region}_{celltype}"


In [16]:
base_name

'AMY_Vascular'

In [17]:
adata = sc.read_h5ad(f"/data2st1/junyi/output/motif/{base_name}.h5ads")

In [18]:
adata

AnnData object with n_obs × n_vars = 181 × 51935
    obs: 'sample', 'sample_barcode', 'obs_names', 'celltype.L2.p', 'celltype.L1.p', 'leiden', 'celltype.L1.GBoost', 'celltype.L1.tab', 'celltype.L1.new', 'expriment'
    uns: 'log1p', 'rank_genes_groups'
    obsm: 'X_umap'
    layers: 'raw'

In [19]:
sc.pp.filter_genes(adata, min_cells=3)
sc.pp.filter_cells(adata, min_genes=200)


In [20]:
adata.var.index=adata.var.index.str.replace(":", "_").str.replace("-", "_")

In [21]:
# Column to use for stratification
if adata.shape[0] > 10000:
    stratify_column = 'sample'
    stratify_column = 'sample'

    # Number of cells to sample from each group
    n_samples_per_group = 2000

    # Perform stratified sampling
    sampled_indices = (
        adata.obs
        .groupby(stratify_column, group_keys=False)
        .apply(lambda x: x.sample(min(n_samples_per_group, len(x))))
        .index
    )
    adata  = adata[sampled_indices, :]



In [22]:
%%capture
adata = ci.add_region_infos(adata)


In [23]:
%%capture
ci.compute_atac_network(
    adata, #metacells,
    organism="mouse",
)

In [24]:
%%capture
final_score = ci.sliding_graphical_lasso(
    adata,
    n_samples=50,
    n_samples_maxtry=100,
    max_alpha_iteration=500,
    verbose=True
)
adata.varp['atac_network'] = final_score


In [25]:
adata.varp['atac_network']

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 513840 stored elements and shape (42404, 42404)>

In [26]:
circe_network = ci.extract_atac_links(adata) #metacells)
circe_network.head(10)


,Peak1,Peak2,score
0,chr14_34602469_34602970,chr14_34616598_34617099,0.830381
1,chr2_5962411_5962912,chr2_6172643_6173144,0.825758
2,chr2_98492392_98492893,chr2_98497262_98497763,0.807276
3,chr2_98496428_98496929,chr2_98497262_98497763,0.778543
4,chr6_82679676_82680177,chr6_82810038_82810539,0.768455
5,chr4_118368508_118369009,chr4_118405568_118406069,0.767991
6,chr5_125459976_125460477,chr5_125460568_125461069,0.764465
7,chr2_98492392_98492893,chr2_98496428_98496929,0.752069
8,chr9_60523545_60524046,chr9_60524158_60524659,0.732264
9,chr9_116794661_116795162,chr9_116896712_116897213,0.731914


In [ ]:
circe_network.to_csv(f"/data2st1/junyi/output/cicre/{base_name}_circe_network.csv")

In [28]:
ccans = ci.find_ccans(circe_network, seed=0)


Coaccessibility cutoff used: 0.07
Number of CCANs generated: 1732


In [ ]:
ccans.to_csv(f"/data2st1/junyi/output/cicre/{base_name}_ccans.csv")

,Peak,CCAN
0,chr2_98496428_98496929,2
1,chr2_98492392_98492893,2
2,chr2_98495223_98495724,2
3,chr2_98497262_98497763,2
4,chr4_118117978_118118479,3
...,...,...
15352,chr1_180716148_180716649,3805
15353,chr1_180764258_180764759,3805
15354,chr1_180978275_180978776,3805
15355,chr1_180794048_180794549,3805


In [30]:
adata = ci.add_ccans(adata)


Coaccessibility cutoff used: 0.07


In [31]:
adata.var.head()


,n_cells,chromosome,start,end,CCAN
chr1_3495191_3495692,3,chr1,3495191,3495692,None
chr1_3741601_3742102,5,chr1,3741601,3742102,None
chr1_4496955_4497456,3,chr1,4496955,4497456,None
chr1_4614183_4614684,4,chr1,4614183,4614684,None
chr1_4759284_4759785,3,chr1,4759284,4759785,None
